In [2]:
import numpy as np
import random as r
from engine import Tensor
import nn

In [3]:
a = Tensor([[1.,2,3],[4,5,6]])
print(a)
w = nn.Linear(3, 1)
print(w.weight)
print(w.bias)
b = w(a)
print(b)

Tensor:
([[1. 2. 3.]
 [4. 5. 6.]])
Grad:
([[0. 0. 0.]
 [0. 0. 0.]])
Tensor:
([[ 0.34489883]
 [-0.36235954]
 [-0.1746805 ]])
Grad:
([[0.]
 [0.]
 [0.]])
Tensor:
([[0.0438577]])
Grad:
([[0.]])
Tensor:
([[-0.86000405]
 [-1.43642767]])
Grad:
([[0.]
 [0.]])


In [4]:
print(w)

Weight:
(Tensor:
([[ 0.34489883]
 [-0.36235954]
 [-0.1746805 ]])
Grad:
([[0.]
 [0.]
 [0.]]))
Bias:
(Tensor:
([[0.0438577]])
Grad:
([[0.]]))
